## <center> Pipelines </center>

Pipelines are a simple way to keep your data preprocessing and modeling code organized. Specifically, a pipeline bundles preprocessing and modeling steps so you can use the whole bundle as if it were a single step.

### Example Melbourne housing

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
# Read the data
data = pd.read_csv('melb_data.csv')

# Separate target from predictors
y = data.Price
X = data.drop(columns="Price")

In [3]:
# Divide data into training and validation subsets
X_train_full, X_test_full, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)


# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality, the value has to be big enough to have a varied but
# non-heterogeneous dataset (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and
                    isinstance(X_train_full[cname].dtype, object)]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if isinstance(X_train_full[cname].dtype, (int, float))]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

In [4]:
X_train.head()

,Rooms,Type,Method,Bathroom,Regionname
12167,1,u,S,1.0,Southern Metropolitan
6524,2,h,SA,2.0,Western Metropolitan
8413,3,h,S,1.0,Western Metropolitan
2919,3,u,SP,1.0,Northern Metropolitan
6043,3,h,S,1.0,Western Metropolitan


#### Step 1: Define Preprocessing Steps

Similar to how a pipeline bundles together preprocessing and modeling steps, we use the ColumnTransformer class to
bundle together different preprocessing steps. The code below:
   - imputes missing values in numerical data
   - imputes missing values and applies a one-hot encoding to categorical data.

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.ensemble import RandomForestRegressor

In [6]:
# Preprocessing for numerical data
numerical_transformer = Pipeline(steps=[('imputer', IterativeImputer(
    estimator=RandomForestRegressor()
))])  # In this case constant can be changed to mean, median, mode, etc...

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

#### Step 2: Define the Model
Next, we define a random forest model with the familiar RandomForestRegressor class.



In [7]:
model = RandomForestRegressor(n_estimators=100, random_state=0)

#### Step 3: Create and Evaluate the Pipeline
Finally, we use the Pipeline class to define a pipeline that bundles the preprocessing and modeling steps. There are a few important things to notice:

- With the pipeline, we preprocess the training data and fit the model in a single line of code. (In contrast, without a pipeline, we have to do imputation, one-hot encoding, and model training in separate steps. This becomes especially messy if we have to deal with both numerical and categorical variables!)
- With the pipeline, we supply the unprocessed features in X_valid to the predict() command, and the pipeline automatically preprocesses the features before generating predictions. (However, without a pipeline, we have to remember to preprocess the validation data before making predictions.)


In [8]:
# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)  
                              ])

# Preprocessing of training data, fit model
my_pipeline.fit(X_train, y_train)

# Evaluate the model
R2 = my_pipeline.score(X_test, y_test)
print('R2:', R2 * 100)

R2: 54.421122412465685
